In [165]:
import tkinter as tk
from tkinter import filedialog, Text, simpledialog, messagebox, ttk
from datetime import datetime
import csv
import pandas as pd
import numpy as np
import requests

In [ ]:
root = tk.Tk()

   
canvas  = tk.Canvas(root, height = 700 , width = 700, bg = "#263D42")
canvas.pack()

#Frame for treeviw Data Frame
frame1 = tk.LabelFrame(root, text = "Data Frame")
frame1.place(height = 350, width = 700)

tv1 = ttk.Treeview(frame1)
tv1.place(relheight = 1 , relwidth = 1)

treescrolly = tk.Scrollbar(frame1 , orient = "vertical", command = tv1.yview)
treescrollx = tk.Scrollbar(frame1 , orient = "horizontal", command = tv1.xview)

tv1.configure(xscrollcommand = treescrollx.set, yscrollcommand = treescrolly.set)
treescrollx.pack(side = "bottom", fill = "x")
treescrolly.pack(side = "right", fill = "y")




# Frame for Status
frame2 = tk.LabelFrame(root, text = "Get Status")
frame2.place(height = 350, width = 700, x = 0, y = 440)

tv2 = ttk.Treeview(frame2)
tv2.place(relheight = 1 , relwidth = 1)

treescrolly2 = tk.Scrollbar(frame1 , orient = "vertical", command = tv2.yview)
treescrollx2 = tk.Scrollbar(frame1 , orient = "horizontal", command = tv2.xview)

tv2.configure(xscrollcommand = treescrollx2.set, yscrollcommand = treescrolly2.set)
treescrollx2.pack(side = "bottom", fill = "x")
treescrolly2.pack(side = "right", fill = "y")

# Buttons
button_frame = tk.LabelFrame(root, text = "Buttons")
button_frame.place (height = 100, width = 700, rely = 0.48, relx =0)

button1 = tk.Button(button_frame, text = "Choose File", command = lambda: file_dialog())
button1.place(relx = 0.05, rely = 0.1)

button2 = tk.Button(button_frame, text = "Load File", command = lambda: load_file())
button2.place(relx = 0.20, rely = 0.1)


button3 = tk.Button(button_frame, text = "Get Status", command = lambda: get_status())
button3.place(relx = 0.35, rely = 0.1)

button4 = tk.Button(button_frame, text = "Add URL", command = lambda: adding())
button4.place(relx = 0.50, rely = 0.1)

button5 = tk.Button(button_frame, text = "Delete URL", command = lambda: deleting())
button5.place(relx = 0.65, rely = 0.1)

button5 = tk.Button(button_frame, text = "Update ", command = lambda: updating())
button5.place(relx = 0.80, rely = 0.1)



label_file = ttk.Label(button_frame, text = "No File Selected")
label_file.place(relx = 0.05 , rely = 0.4)


#### Functions

def file_dialog():
    filename = filedialog.askopenfilename(initialdir = "/",
                                          title = "Select A File",
                                          filetypes = (("executable", "*.csv"), ("All files","*.*")))
    label_file["text"] = filename
    return None


def load_file():
    file_path = label_file["text"]
    try:
        csv_file = r"{}".format(file_path)
        df = pd.read_csv(csv_file, index_col=[0])
    except ValueError:
        tk.messagebox.showerror("information", "The file you have chosen is invalid")
        return None
    except FileNotfoundError:
        tk.messagebox.showerror("information", f"No such file as {file_path}")
        return None
    clear_data()
    tv1["column"] = list(df.columns)
    tv1["show"] = "headings"
    for column in tv1["columns"]:
        tv1.heading(column, text = column)
            
    df_rows = df.to_numpy().tolist()
    for row in df_rows:
        tv1.insert("", "end", values = row)
    return None

def clear_data():
    tv1.delete(*tv1.get_children())
def clear_data2():
    tv2.delete(*tv2.get_children())

def status(website):
    r = requests.get(website)
    if r.status_code>= 300:
        return "FAIL"
    else: 
        return "OK"
    
    
def get_status():
    file_path = label_file["text"]
    try:
        csv_file = r"{}".format(file_path)
        df = pd.read_csv(csv_file,index_col=[0])
    except ValueError:
        tk.messagebox.showerror("information", "The file you have chosen is invalid")
        return None
    except FileNotFoundError:
        tk.messagebox.showerror("information", f"No such file as {file_path}")
        return None
    clear_data2()
    tv2["columns"] = ('Website', 'Status')
    tv2.column('#0', width=0)
    tv2.column('Website', width=100)
    tv2.column('Status', width=100)
    tv2.heading('Website', text='Website')
    tv2.heading('Status', text='Status')
    i=0
    for web in df["Address"]:
        tv2.insert(parent='', index=i, text='', values=(web, status(web)))
        i +=1   
    tv2.pack()
    
    
###### Adding new URLs

def adding():
    file_path = label_file["text"]
    try:
        csv_file = r"{}".format(file_path)
        df = pd.read_csv(csv_file,index_col=[0])
    except ValueError:
        tk.messagebox.showerror("information", "The file you have chosen is invalid")
        return None
    except FileNotFoundError:
        tk.messagebox.showerror("information", f"No such file as {file_path}")
        return None
    USER_INP = simpledialog.askstring(title="Adding Website",
                                  prompt="what is the address?")
    df2 = pd.DataFrame({"Address": [USER_INP],
                    "Situation": [status(USER_INP)],
                    "Time_added": [datetime.now()],
                    "Last_update": [datetime.now()]})
    
    df.append(df2).to_csv(file_path)    



###### Deleting a URL
def deleting():
    file_path = label_file["text"]
    try:
        csv_file = r"{}".format(file_path)
        df = pd.read_csv(csv_file,index_col=[0])
    except ValueError:
        tk.messagebox.showerror("information", "The file you have chosen is invalid")
        return None
    except FileNotFoundError:
        tk.messagebox.showerror("information", f"No such file as {file_path}")
        return None
    USER_INP = simpledialog.askstring(title="Deleting Website",
                                  prompt="what is the address?")
    df.loc[df["Address"] != USER_INP].to_csv(file_path)    


    
########## Updating the dataframe    
def updating():
    file_path = label_file["text"]
    try:
        csv_file = r"{}".format(file_path)
        df = pd.read_csv(csv_file,index_col=[0])
    except ValueError:
        tk.messagebox.showerror("information", "The file you have chosen is invalid")
        return None
    except FileNotFoundError:
        tk.messagebox.showerror("information", f"No such file as {file_path}")
        return None
    clear_data()
    situation = []
    for i in df["Address"]:
        situation.append(status(i))
    df["status"] = situation
    df["Last_update"] = datetime.now()   
    df.to_csv(file_path)
    
    
    
root.mainloop()